# PlayTimeGenre EDA y Tranformacion

In [78]:
# importacion de librerias
import pandas as pd
from datetime import datetime
import html

Lectura de los datasets necesarios para el endpoint

* users_items.csv: solo se cargan las columnas 'item_id' y 'playtime_forever'
* steam_game.csv: solo se cargan las columas 'genres', 'release_date' y 'id'

In [79]:
df_users_item = pd.read_csv("../csv/users_items.csv", sep=",", usecols=['item_id', 'playtime_forever'])
df_steam_game = pd.read_csv("../csv/steam_game.csv", sep=",", usecols=['genres', 'release_date', 'id'], parse_dates=["release_date"], date_format='%Y-%m-%d')

## Procesamiento de los users_items

Se agrupan por item_id y se sumariza el playtime_forever

In [82]:
df_juegos_playtime = df_users_item.groupby(by='item_id',as_index=False)['playtime_forever'].sum()
df_juegos_playtime.head()

,item_id,playtime_forever
0,10,17107858
1,20,960524
2,30,756375
3,40,154424
4,50,726545


In [87]:
#se renombra la comlumna para futuro merge
df_juegos_playtime.rename(columns={'item_id':'id','playtime_forever':'playtime'},inplace=True)

In [88]:
#no hay nulos
df_juegos_playtime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10978 entries, 0 to 10977
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   id        10978 non-null  int64
 1   playtime  10978 non-null  int64
dtypes: int64(2)
memory usage: 171.7 KB


## Procesamiento de steam_game

Se agrupan por item_id y se sumariza el playtime_forever

In [89]:
# se detectan nulos, si alguna fila tiene algún nulo, la fila no es relevante, se debe eliminar
df_steam_game.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32132 entries, 0 to 32131
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28850 non-null  object
 1   release_date  30066 non-null  object
 2   id            32132 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 753.2+ KB


In [90]:
#eliminar filas que contengan alguna columna en null no son aptas para la funcionalidad
df_steam_game.dropna(inplace = True)
df_steam_game.shape

(28832, 3)

Se realiza el merge de los datframes a partir del id

el inner join es adecuado ya que valores nulos en el resultado no serían relevantes

In [91]:
df_play_time_genre = pd.merge(df_steam_game, df_juegos_playtime, on='id')
df_play_time_genre.head()

,genres,release_date,id,playtime
0,"['Action', 'Indie', 'Racing']",1997-06-30,282010,9319
1,['Action'],1998-11-08,70,2650946
2,['Strategy'],2006-07-06,1640,27162
3,['Strategy'],2006-07-06,1630,21105
4,"['Action', 'Indie', 'RPG']",2006-07-11,2400,1030947


In [92]:
#no se detectan nulos
df_play_time_genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8653 entries, 0 to 8652
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        8653 non-null   object
 1   release_date  8653 non-null   object
 2   id            8653 non-null   int64 
 3   playtime      8653 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 270.5+ KB


In [103]:
#se detectan fechas con diferentes formatos
df_play_time_genre[df_play_time_genre["release_date"].str.len() != 10]["release_date"]

181              Jun 2009
373              Oct 2010
376              Oct 2010
377              Oct 2010
424              Feb 2011
1279             Sep 2014
1742             Apr 2015
2938             Apr 2016
3516    Not yet available
3566             Apr 2017
3571        Coming soon..
3602         Coming Soon!
3656             Oct 2016
3981             Jul 2016
4919                  TBD
5520             Aug 2015
5888             Jun 2015
5973             May 2015
6486             Jan 2015
6853             Aug 2014
6991             Jul 2014
7117             May 2014
7782             Dec 2012
8205             Mar 2010
8278             Oct 2009
8320             Sep 2009
Name: release_date, dtype: object

In [56]:
#se define una funcion para obtner el año
def obtner_anio(cadena):
    anio = None
    fecha = cadena.split('-')
    mes_anio = cadena.split(' ')
    if len(fecha) == 3:
        anio = fecha[0]
    elif len(mes_anio) == 2 and len(mes_anio[1]) == 4:
        anio = mes_anio[1]
    return anio


In [104]:
# flatenizar, normalizar, escapar transformar entidades html

data = []
for indice, fila in df_play_time_genre.iterrows():
    anio = obtner_anio(fila['release_date'])
    playtime = fila['playtime']
    genres = fila['genres'].strip('[]').split(',')
    for genre in genres:
        data.append({'genre':html.unescape(genre.replace("'","").lower()),'anio': anio, 'playtime': playtime})     
    
df_play_time_genre_flat = pd.DataFrame(data)
#dimensiones
df_play_time_genre_flat.head()

,genre,anio,playtime
0,action,1997,9319
1,indie,1997,9319
2,racing,1997,9319
3,action,1998,2650946
4,strategy,2006,27162


In [105]:
df_play_time_genre_flat.shape

(21545, 3)

In [106]:
#agrupar por genero y año, y sumarizar el tiempo jugado
ptg_sum = df_play_time_genre_flat.groupby(['genre','anio'], as_index=False)['playtime'].sum()
ptg_sum.head()


,genre,anio,playtime
0,action,2008,1058
1,action,2012,1978255
2,adventure,1983,3473
3,adventure,1984,384
4,adventure,1988,16001


In [68]:
ptg_sum.shape

(545, 3)

In [108]:
#se guarda como csv para render
ptg_sum.to_csv(r'data/play_time_genre.csv',index=False)

# Modelo de recomendación ML

Se programa un sistema de recomendación ítem-ítem.

Se usarán palabras correspondientes al dataset steam_game, con la vectorizacion de las palabras se creará una matriz de similitud de coseno.

Finalemnte se exportrá un documento que contenga  5 recomendaciones para cada juego.

In [ ]:
# Se importan las librerías necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Se leen las columnas importantes del dataset procesado durante el ETL

df_steam_game = pd.read_csv("../csv/steam_game.csv", sep=",", usecols=['publisher','genres','app_name','tags','specs', 'developer', 'id'])
df_steam_game.head()

In [ ]:
#Se eliminan filas con nulos
df_steam_game.dropna(inplace=True)
df_steam_game.shape

In [ ]:
#exploracion de la columna publisher
df_steam_game.publisher.value_counts()

In [ ]:
#exploracion de la columna genres
df_steam_game.genres.value_counts()

In [ ]:
#se normalizan y limpian datos de columnas de texto importantes para la matriz de similitud

df_steam_game["genres"]=[str(genre).strip('[]').replace("'","").replace(",","").replace("nan","") for genre in df_steam_game["genres"]]
df_steam_game["tags"]=[str(tag).strip('[]').replace("'","").replace(",","").replace("nan","") for tag in df_steam_game["tags"]]
df_steam_game["specs"]=[str(spec).strip('[]').replace("'","").replace(",","").replace("nan","") for spec in df_steam_game["specs"]]
df_steam_game["publisher"]=[str(pub).replace(",","").replace("nan","") for pub in df_steam_game["publisher"]]
df_steam_game["developer"]=[str(dev).replace(",","").replace("nan","") for dev in df_steam_game["developer"]]

In [ ]:
# se define una funcion que unificará las columnas de texto, juntar todas las palabras en una columna, llamada "imp"
def important_features(dataset):
    data=dataset.copy()
    imp = []
    for indice, fila in data.iterrows():        
        imp.append(str(fila["publisher"])+' '+str(fila["genres"])+" "+str(fila["app_name"])+" "+str(fila["tags"])+" "+str(fila["specs"])+" "+str(fila["developer"]))
    data["imp"] = pd.DataFrame(imp)
    return data

In [ ]:
# se guarda el nuevo dataset en data
data=important_features(df_steam_game)

In [ ]:
#se guarda el dataset como csv ante un eventual crasheo de python 
data.to_csv(r'data/recomendacion_data.csv',index=False)

In [ ]:
# se importa TfidfVectorizer

vec=TfidfVectorizer()

#se vetorizan las palabras de las features importantes, a media que se las comvierte de string de numpy

vecs=vec.fit_transform(data["imp"].apply(lambda x: np.str_(x)))

In [ ]:
#se calcula la matriz de similitud, se tiene tantas filas y columnas como juegos
similarity =cosine_similarity(vecs) 

In [ ]:
#se define una fucion que recibe el juego_id, se busca el index de ese juego en el dataset
# se obtiene el vector de similitudes, se lo ordena de mayor a menor y se toman los 5 primeros valores, sin contar el juego mismo
#con los indices de la matriz de simulitud se buscan los juegos y se guarda el nombre

def recommend(id):    
    game_idx=data[data["id"] == id].index.values[0]
    if game_idx < similarity.shape[0]:
        scores=list(enumerate(similarity[game_idx]))
        sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
        sorted_scores=sorted_scores[1:6]
        games=[data.iloc[game[0]]["app_name"].replace('"','') for game in sorted_scores]
        return games
    else:
        return []

In [ ]:
#finalmente se arma un dataset que contiene todos los juegos y las 5 sugerencias para cada uno
# cual sera consumido por la api para responder a las peticiones

recom = []
for indice, fila in data.iterrows():
    recom.append({'game_id':fila.id,'recom':recommend(fila.id)})
    
df_recom = pd.DataFrame(recom)


df_recom.head()



In [ ]:
df_recom.to_csv(r'data/recomendacion_db.csv', index=False, sep=";")